In [ ]:
import ollama 
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader 
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Workouts.pdf")
pages = loader.load_and_split()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

In [ ]:
embeddings = OllamaEmbeddings(model="zephyr")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
def format_docs (docs) :
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
def ollama_llm(question, context) :
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama. chat(model='zephyr', messages=[{'role': 'user','content' : formatted_prompt}])
    return response['message'] ['content']

In [ ]:
def rag_chain(question) :
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [ ]:
answer = rag_chain("I want a Mindful workout")

In [ ]:
print(answer)